In [20]:
import pandas as pd
from striprtf.striprtf import rtf_to_text
import re
import os
import locale
from datetime import datetime
locale.setlocale(locale.LC_TIME, "nl_NL")
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [34]:
plain_texts = []
folder_path = "data"  ### There needs to be folder in the working directory with the title "data" that has separated rtf.file
file_names = os.listdir(folder_path)
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, "r", encoding = 'latin1') as file:
        text = file.read()
        text = rtf_to_text(text)
        plain_texts.append(text)

In [36]:
dct = {}
for index, t in enumerate(plain_texts): 
    dct[str(index)] = {}
    
    meta = re.search(r"^(.*?)Copyright", t, re.DOTALL) #### Can/must be checked for all variables
    section = re.search(r"Section:(.*?)\n", t)
    length = re.search(r"Length:(.*?)\n", t)
    byline = re.search(r"Byline:(.*?)\n", t)
    highlight = re.search(r"Highlight:(.*?)\n", t)
    body = re.search(r"Body(.*?)Load-Date:", t.replace("\n", " "))
    
    if meta:
        meta_data = meta.group(1).strip().split("\n")
        dct[str(index)]["headline"] = meta_data[0]
        dct[str(index)]["news_source"] = meta_data[1]
        dct[str(index)]["datePublished"] = meta_data[2]
        if len(meta_data) > 3:
            dct[str(index)]["region"] = meta_data[3]
    if section:
        dct[str(index)]["articleSection"] = section.group(1).strip()
    if length:
        dct[str(index)]["length"] = length.group(1).strip()
    if byline:
        dct[str(index)]["byline"] = byline.group(1).strip()
    if highlight:
        dct[str(index)]["highlight"] = highlight.group(1).strip()
    if body:
        dct[str(index)]["text"] = body.group(1).strip()

In [38]:
df_print = pd.DataFrame(dct).transpose()

df_print

,headline,news_source,datePublished,region,articleSection,length,byline,highlight,text
0,"Transition écologique A Muttersholtz, les bien...",,Libération,jeudi 14 mars 2024,STORIES; Pg. 12,1800 words,Olivier MonodJulie Renson MiquelPascal Bastien...,"Autoproduction énergétique, lutte contre l'art...",«La hausse des prix de l'énergiene pèse pas su...
1,"CYCLISME Christian Prudhomme, un roi bien en ...",,Libération,jeudi 13 juillet 2023,STORIES; Pg. 19,2026 words,Romain BoulhoPascal Aimar. Tendance floue; Rom...,"Chef d'entreprise, figure patrimoniale, élu en...",L'alerte fut donnée début juillet sur Facebook...
2,Les visiteurs mis à contribution pour protég...,Le Figaro,Samedi 8 Juillet 2023,NaN,SOCIÉTÉ; Pg. 11; N° 24534,811 words,"Taupin, Astrid",NaN,"ÉROSION, pollution, flore dégradée... La liste..."
3,"TROYES, LA PETITE CITÉ CHAMPENOISE QUI PÉTILLE...",Le Figaro,Vendredi 17 Février 2023,NaN,SCOPE; Pg. 37; N° 24414,1993 words,"Carillet, Jean-Bernard",NaN,C'est comme si elle avait retrouvé tout son éc...
4,La capitale asphyxiée par les travaux; Par mil...,Le Figaro,Samedi 15 Juin 2019,NaN,SOCIÉTÉ; Pg. 10; N° 23276,688 words,"Négroni, Angélique",NaN,URBANISME Alors que le trafic automobile ne c...
...,...,...,...,...,...,...,...,...,...
877,"LES ROLLING STONES, DIEUX DU STADE; APRÈS LOND...",Le Figaro,Vendredi 11 Juin 2021,NaN,CULTURE; &vous; Pg. 35; N° 23889,671 words,"Nuc, Olivier",NaN,Malgré la pandémie qui prive les rock stars in...
878,Travelling sur l'attentat de Sarajevo; VELIBOR...,Le Figaro,Jeudi 31 Mai 2012,NaN,CRITIQUE; Pg. 2; N° 21097,423 words,"Dargent, Françoise",NaN,ON N'IGNORE rien des effets de l'attentat de S...
879,Incendie à Vaulx-en-Velin : «On veut savoir co...,,Libération,samedi 17 décembre 2022,EXPRESSO; Pg. 12,882 words,"Maïté Darnault,; Maïté Darnault, Envoyée spéci...",L'origine du drame qui a fait dix morts dont c...,Ce vendredi aurait dû être un prélude paisible...
880,"TRANSPORTS En un an, il a conquis 200 000 abon...",Le Figaro,Mercredi 25 Juin 2008,NaN,RECTO; Verso; Pg. 2; N° 19875,1240 words,"Tabet, Marie-Christine",NaN,C'est un coup marketing incontestable. Un de c...


In [40]:
df_print.to_csv("dataset2.csv", encoding='utf-8', index=False)

